In [143]:
import os

def load_data(filepath):
    """Charge un fichier et retourne une liste de blocs (groupes de 11 lignes)."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Séparer les blocs de 11 lignes
        blocks = []
        current_block = []

        for line in lines:
            stripped_line = line.strip()

            # Ajouter la ligne au bloc actuel si elle n'est pas vide
            if stripped_line:
                current_block.append(stripped_line)

            # Lorsque le bloc atteint 11 lignes, on l'ajoute et on réinitialise
            if len(current_block) == 11:
                blocks.append("\n".join(current_block))
                current_block = []

        return blocks

    except FileNotFoundError:
        print(f"Erreur : Le fichier {filepath} est introuvable.")
        return []
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {filepath} : {e}")
        return []


def classify_and_save_blocks(blocks, classification, train_file, test_file):
    """
    Classifie les blocs en train et test, ajoute la classification et les sauvegarde.
    """
    for block in blocks:
        # Extraire la première ligne du bloc
        first_line = block.split("\n")[0].strip()

        # Vérifier le type de bloc
        if first_line.startswith('*'):  # Train
            file = train_file
        else:  # Test
            file = test_file

        # Ajouter le bloc et la classification au fichier
        file.write(block + "\n\n")
        file.write(str(classification) + "\n\n")


def save_datasets(mine_file, rocks_file, train_filepath, test_filepath):
    """Gère la séparation des blocs et leur classification."""
    try:
        # Charger les fichiers
        mine_blocks = load_data(mine_file)
        rocks_blocks = load_data(rocks_file)

        # Ouvrir les fichiers de sortie
        os.makedirs(os.path.dirname(train_filepath), exist_ok=True)
        with open(train_filepath, 'w', encoding='utf-8') as train_file, \
             open(test_filepath, 'w', encoding='utf-8') as test_file:

            # Classifier et sauvegarder les blocs des fichiers "mines" et "rocks"
            classify_and_save_blocks(mine_blocks, 1, train_file, test_file)  # Mines : classification = 1
            classify_and_save_blocks(rocks_blocks, -1, train_file, test_file)  # Rocks : classification = -1

    except Exception as e:
        print(f"Erreur lors de la création des datasets : {e}")


# Chemins vers les fichiers sources
mine_file = 'sonar_data/sonar.mines'
rocks_file = 'sonar_data/sonar.rocks'

# Chemins des fichiers de sortie
train_filepath = 'tp2/result_sets/train_set.txt'
test_filepath = 'tp2/result_sets/test_set.txt'

# Générer les datasets
save_datasets(mine_file, rocks_file, train_filepath, test_filepath)

print("Les fichiers train_set.txt et test_set.txt ont été créés avec succès !")


Les fichiers train_set.txt et test_set.txt ont été créés avec succès !


In [144]:
from classes.ensemble import Ensemble
from classes.paire import Paire

def lire_fichier_ensemble(filepath):
    """
    Lit un fichier train ou test et crée un objet Ensemble.

    Args:
        filepath (str): Le chemin vers le fichier à lire.

    Returns:
        Ensemble: Un objet contenant toutes les paires lues depuis le fichier.
    """
    ensemble = Ensemble()
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = [line.strip() for line in file if line.strip()]  # Supprime les lignes vides
            i = 0

            while i < len(lines):
                bloc_donnees = []

                # Lire les lignes jusqu'à rencontrer une ligne vide ou un en-tête
                while i < len(lines) and lines[i] and not lines[i].startswith('*') and not lines[i].startswith('CM'):
                    bloc_donnees.append(lines[i])
                    i += 1

                # Si on n'a pas assez de lignes pour former un bloc valide, on arrête
                if len(bloc_donnees) < 12:
                    break

                # La 13ème ligne contient la classification
                classification_str = lines[i].strip()
                if classification_str.startswith('*') or classification_str.startswith('CM'):
                    continue  # Ignorer les en-têtes de bloc

                try:
                    classification = int(classification_str)
                    i += 1
                except ValueError:
                    print(f"Erreur : Classification invalide pour le bloc : {classification_str}")
                    continue

                # Convertir les données en un vecteur de floats en combinant toutes les lignes du bloc
                vecteur = []
                for data_line in bloc_donnees:
                    vecteur.extend([float(val) for val in data_line.strip('{}').split(',')])

                # Créer une paire et l'ajouter à l'ensemble
                paire = Paire(vecteur, classification)
                print(paire)
                ensemble.ajouter_paire(paire)

                # Sauter la ligne vide avant de passer au bloc suivant
                i += 1
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {filepath} : {e}")
    return ensemble

# Exemple d'utilisation
train_ensemble = lire_fichier_ensemble('tp2/result_sets/train_set.txt')
test_ensemble = lire_fichier_ensemble('tp2/result_sets/test_set.txt')

# Affichage pour validation
print("Train Ensemble contient :", len(train_ensemble.elements), "paires.")
print("Test Ensemble contient :", len(test_ensemble.elements), "paires.")


Train Ensemble contient : 0 paires.
Test Ensemble contient : 0 paires.
